In [1]:
import pyhf
import json
import numpy as np
import scipy
import sys
np.set_printoptions(threshold=sys.maxsize)

import simplify # need a local dev version for this

pyhf.set_backend(pyhf.tensorlib, "minuit")
spec = json.load(open("BkgOnly.json", "r"))

ws = pyhf.Workspace(spec)
model = ws.model(modifier_settings = {"normsys": {"interpcode": "code4"},"histosys": {"interpcode": "code4p"},})
data = ws.data(model)

In [2]:
# run fit
fit_result = simplify.fitter.fit(model, data)

# uncertainties for later
total_stdev_model = simplify.model_tools.calculate_stdev(
    model, fit_result.bestfit, fit_result.uncertainty, fit_result.corr_mat
)

# correlation matrix for now
plt = simplify.plot.correlation_matrix(fit_result,"",pruning_threshold=0.2)

In [13]:
eigVals, eigVecs = np.linalg.eig(fit_result.corr_mat) # real symmetric matrix, always diagonalisable, yeay

# eigendecomposition: A = Q*Lambda*Q^T, where Lambda is diag(eVals), Q is matrix with eVecs as columns
eigVals_diag = np.diag(eigVals)
eigVecs.dot(eigVals).dot(eigVecs.T) # this gives us the original corrmatrix back, right?

# May want to sort them?
# _order = np.argsort(eigVals)
# eigVals[_order]

m = np.linalg.inv(eigVals_diag.dot(eigVecs.T))

#this should be a unit matrix
np.testing.assert_allclose(
    eigVals_diag.dot(eigVecs.T).dot(m),
    np.identity(len(fit_result.labels)),
    atol=1e-12
)

# this should give us back the matrix with EV as columns
np.testing.assert_allclose(
    fit_result.corr_mat.dot(m),
    eigVecs,
    atol=1e-3 # why is this so bad?
)

bestfit_eigenspace = fit_result.bestfit.dot(m)
uncertainty_eigenspace = np.sqrt(np.square(fit_result.uncertainty).dot(np.abs(m)))

# Sometimes nice to see what the actual values are
# for l, b in zip(fit_result.labels, fit_result.bestfit):
#     print(l,b)

# generate new fit result object for hacking simplify plotting 
labels = [f"Effective NP {i+1}" for i in range(len(bestfit_eigenspace))]
result_eigen = simplify.fitter.FitResults(bestfit_eigenspace, uncertainty_eigenspace, labels, fit_result.types, fit_result.cov_mat,  eigVals_diag, fit_result.best_twice_nll)

In [17]:
import pathlib

def eigenvalue_matrix(
    fit_results,
    output_path,
    pruning_threshold,
    **kwargs: int,
) -> None:

    # pruning all entries below threshold with mask
    below_threshold = np.where(
        np.abs(fit_results.corr_mat) < pruning_threshold, True, False
    )
    all_below_threshold = np.all(below_threshold, axis=0)
    fixed_parameter = np.all(np.equal(fit_results.corr_mat, 0.0), axis=0)

    delete_indices = np.where(np.logical_or(all_below_threshold, fixed_parameter))
    corr_mat = np.delete(
        np.delete(fit_results.corr_mat, delete_indices, axis=1), delete_indices, axis=0
    )
    labels = np.delete(fit_results.labels, delete_indices)
    figure_path = pathlib.Path(output_path) / "eigenvalue_matrix.pdf"

    # borrowing correlation matrix code for this
    simplify.helpers.plotting.correlation_matrix(corr_mat, labels, figure_path, **kwargs)

plt = eigenvalue_matrix(result_eigen,"",pruning_threshold=1.1, **{'vmin':0.1, 'vmax':4, 'cmap':'Blues'})

In [137]:
import awkward1 as ak
def calculate_uncertainty(
    model,
    parameters,
    uncertainty,
    corr_mat,
):

    # indices where to split to separate all bins into regions
    region_split_indices = simplify.model_tools._get_channel_boundary_indices(model)

    up_variations = []
    down_variations = []
    for i_par in range(model.config.npars):
        # central parameter values, but one parameter varied within uncertainties
        up_pars = parameters.copy()
        up_pars[i_par] += uncertainty[i_par]
        down_pars = parameters.copy()
        down_pars[i_par] -= uncertainty[i_par]

        # total model distribution with this parameter varied up
        up_combined = model.expected_data(up_pars, include_auxdata=False)
        up_yields = np.split(up_combined, region_split_indices)
        up_variations.append(up_yields)

        # total model distribution with this parameter varied down
        down_combined = model.expected_data(down_pars, include_auxdata=False)
        down_yields = np.split(down_combined, region_split_indices)
        down_variations.append(down_yields)

    # convert to awkward arrays for further processing
    up_variations = ak.from_iter(up_variations)
    down_variations = ak.from_iter(down_variations)

    # total variance, indices are: channel, bin
    total_variance_list = [
        np.zeros(shape=(model.config.channel_nbins[ch])) for ch in model.config.channels
    ]  # list of arrays, each array has as many entries as there are bins
    total_variance = ak.from_iter(total_variance_list)

    # loop over parameters to sum up total variance
    # first do the diagonal of the correlation matrix
    for i_par in range(model.config.npars):
        symmetric_uncertainty = (up_variations[i_par] - down_variations[i_par]) / 2
        total_variance = total_variance + symmetric_uncertainty ** 2

    labels = simplify.model_tools.get_parameter_names(model)

    # convert to standard deviation
    total_stdev = np.sqrt(total_variance)
    return total_stdev

In [139]:
all_uncs_eigenspace = calculate_uncertainty(model,bestfit_eigenspace,uncertainty_eigenspace,np.identity(len(uncertainty_eigenspace)))